In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import plotly.io as pio

pio.renderers.default = "notebook_connected"

import sys
import re

sys.path.append("../../../")
sys.path.append("../../python")
import pandas as pd

pd.set_option("display.max_colwidth", 0)

In [2]:
# If you're reading this: I'm hiding all of the code here in this top cell block 
# so it doesn't show up as ugly & large codeblocks in the rest of the doc.
# For code articles & tutorials we want the code to be present, 
# but since this is an explainer we'd rather hide it. 
# I'm not aware of any good alternatives to hide input but not output, so this will do for now.

import numpy as np
import plotly.express as px
import polars as pl
from string import ascii_uppercase

settingsShowTSSpread=pl.DataFrame({"n": 5, "positives": range(1, 1001)}).with_columns(
    p=[0.01, 0.05, 0.1]
).explode("p").with_columns(evidence=pl.col("positives") / pl.col("p"))

def betaDistribution(structcol):
    return structcol.apply(
        lambda x: np.random.beta(
            x["p"] * x["evidence"], (1 - x["p"]) * x["evidence"], x["n"]
        ).tolist()
    )


thompsonSamplingSimulation = settingsShowTSSpread.with_columns(
    sampled_propensity=betaDistribution(pl.struct(["n", "p", "evidence"]))
).explode('sampled_propensity').with_columns(positives = pl.col('evidence')*pl.col('p'))

# Spread of the Thompson Sampled propensities
ThompsonSamplingSpread = px.scatter(
    thompsonSamplingSimulation.to_pandas(),
    x="positives",
    y="sampled_propensity",
    color="p",
    opacity=0.6,
    labels={
        "sampled_propensity": "Sampled Propensity",
        "positives": "Number of positive responses in the Adaptive Model",
        "p":"Propensity"
    },
    range_y=[0, 0.2],
    title='Thompson Sampling',
    template="none",
).update_coloraxes(showscale=False).update_traces(marker={"size":3}).update_yaxes({'tickformat':',.0%', 'tickmode':'array', 'tickvals':[0, 0.01, 0.05, 0.1]})

# Convergence of the Thompson Sampled propensities
s = thompsonSamplingSimulation['positives']
thompsonSamplingSimulation2 = thompsonSamplingSimulation.hstack(s.cut(breaks=np.array(range(int(s.min()), int(s.max())+20, 20))-1, series=False).select(bin='category'))
s = thompsonSamplingSimulation2.groupby("p", "bin").agg(
    n=pl.count(),
    n90=(((pl.col("sampled_propensity") - pl.col("p")) / pl.col("p")) < 0.1).sum(),
    positives=pl.min("positives"),
).with_columns(pct=pl.col('n90')/pl.col('n')).sort('p', 'bin').with_columns(pl.col('p').cast(pl.Utf8).cast(pl.Categorical))
PercentageOfSampled = px.line(s.to_pandas(), x='positives', y='pct', color='p', template='none', line_group='p', title='Percentage of Sampled Propensities <br><sup>that are within 10% of the Model Propensities</sup>', labels={'pct':'Percentage', 'positives':'Percentage of positive responses in the Adaptive Model', 'p':'Propensity'}).update_layout(yaxis_tickformat='.0%')

# Beta distribution
settings1 = (
    pl.DataFrame({"n": 100000})
    .with_columns(
        p=[0.01, 0.05, 0.1], evidence=[2000, 200, 100000], ypeak=[200, 50, 500])
    .explode("p", "evidence", 'ypeak')
).with_columns(
    sampled_propensity=betaDistribution(pl.struct(["n", "p", "evidence"]))
).explode('sampled_propensity').with_columns(positives = pl.col('evidence')*pl.col('p'))
from scipy.stats import gaussian_kde
results = {}
for p, series in settings1.groupby('p'):
    results[str(p)] = gaussian_kde(series['sampled_propensity'], 'silverman')(np.arange(0,0.15,0.0001))
results = pl.DataFrame(results).with_columns(sampledPropensity=pl.Series(np.arange(0,0.15,0.0001))).to_pandas().set_index('sampledPropensity')
DistributionOfSampled = px.area(results, title='Distribution of the sampled propensities<br><sup>for a few combinations of model propensity and evidence</sup>', template='none', labels={'value':'', 'sampledPropensity':'Sampled Propensity', 'variable':'Propensity'}).update_yaxes({'visible':True}).update_xaxes({'tickformat':',.0%', 'tickmode':'array', 'tickvals':[0, 0.01, 0.05, 0.1]}).update_layout(showlegend=False).update_traces({'line':{'width':0.0}})#.add_annotation()
for annot in list(settings1.select('p', 'ypeak', 'positives').unique().iter_rows(named=True)):
    DistributionOfSampled.add_annotation(x=annot['p'],y=annot['ypeak']+20, text = f"Propensity = {annot['p']:.0%}<br>Positives = {annot['positives']}", bgcolor='#FFFFFF', bordercolor='#000000', showarrow=False)

# Outbound Model Maturity
OutboundMaturity = px.line(
    pl.DataFrame(
        {"Positives": [0, 200, 300], "modelMaturityBasedReach": [0.02, 1.0, 1.0]}
    ).to_pandas(),
    x="Positives",
    y="modelMaturityBasedReach",
    template="none",
    labels={
        "modelMaturityBasedReach": "% Selected from Eligible Customers",
        "Positives": "Number of Positives in the Adaptive Model",
    },
).update_layout(yaxis_tickformat='.0%')

# Simulating Outbound Maturity
nInitialActions = 5
basePropensity = 0.01
ncust = 100000

letters = list(ascii_uppercase)
def simulateActions(initialActions):
    actions = initialActions.with_columns(
        positives=0.0, modelMaturityBasedReach=0.0, expectedaccepts=0.0, impressions=0.0
    )
    for w in range(0, 11):
        if w > 0:
            # Create new action row
            newWeek = pl.DataFrame(
                {
                    "action": letters[nInitialActions + w - 1],
                    "basepropensity": basePropensity,
                    "evidence": 0.0,
                    "week": w,
                    "positives": 0.0,
                    "modelMaturityBasedReach": 0.0,
                    "expectedaccepts": 0.0,
                    "impressions": 0.0,
                },
                schema_overrides={"week": pl.UInt8},
            )
            # Add the new action to the previous week's data
            newWeek = pl.concat(
                [
                    actions.filter(pl.col("week") == w - 1).with_columns(
                        week=pl.lit(w).cast(pl.UInt8)
                    ),
                    newWeek,
                ]
            )
            # Add this new table to the main actions table
            actions = pl.concat([actions, newWeek])

        # Simulate new positives
        actions = actions.with_columns(
            positives=pl.col("evidence") * pl.col("basepropensity")
        )

        # Simulate maturity based reach
        actions = actions.with_columns(
            modelMaturityBasedReach=pl.when(pl.col("positives") >= 200)
            .then(1.0)
            .otherwise(0.02 + (0.98 * (pl.col("positives") / 200))),
        )

        # Calculate expected accepts and impressions for new week
        actions = actions.with_columns(
            [
                pl.when(pl.col("week") == w)
                .then(
                    pl.lit(ncust)
                    * pl.col("modelMaturityBasedReach")
                    * pl.col("basepropensity")
                )
                .otherwise(pl.col("expectedaccepts"))
                .alias("expectedaccepts"),
                pl.when(pl.col("week") == w)
                .then(pl.lit(ncust) * pl.col("modelMaturityBasedReach"))
                .otherwise(pl.col("impressions"))
                .alias("impressions"),
            ]
        ).with_columns(
            pl.when(pl.col("week") == w)
            .then(
                pl.col("impressions")
                * pl.min_horizontal(
                    [
                        pl.lit(1.0),
                        (
                            pl.lit(ncust)
                            / pl.col("impressions").where(pl.col("week") == w).sum()
                        ),
                    ]
                )
            )
            .otherwise(pl.col("impressions"))
            .alias("impressions")
        )

        # Set evidence to the previous week's evidence + impressions
        def previousEvidence():
            return (pl.col("evidence").shift(1).over(["action"])) + (
                pl.col("impressions").shift(1).over("action")
            )

        if w > 0:
            actions = actions.with_columns(
                evidence=pl.when(previousEvidence().is_not_null())
                .then(previousEvidence())
                .otherwise(pl.col("evidence"))
            )

    fig = px.area(
        actions.sort("action", descending=True).to_pandas(),
        x="week",
        y="impressions",
        color="action",
        template="none",
        title="Effect of Outbound Model Maturity <br><sup>when introducing weekly new actions</sup>",
        labels={"impressions": "Number of Customers"},
    ).update_layout(legend={"traceorder": "reversed"})
    scale = px.colors.sample_colorscale(
        "RdBu", [n / (len(fig.data) - 1) for n in range(len(fig.data))]
    )
    for i in range(len(fig.data)):
        fig.data[i]["line"]["color"] = scale[i]
    return fig


initialTab = pl.DataFrame(
    {
        "action": letters[0:nInitialActions],
        "basepropensity": np.random.normal(
            loc=basePropensity, scale=0.001, size=nInitialActions
        ),
        "evidence": np.floor(
            np.random.uniform(size=nInitialActions, low=15000, high=40000)
        ),
        "week": 0,
    },
    schema_overrides={"week": pl.UInt8},
)
WithExistingActions = simulateActions(initialTab)


initialTab = pl.DataFrame(
    {
        "action": letters[0:nInitialActions],
        "basepropensity": np.random.normal(
            loc=basePropensity, scale=0.001, size=nInitialActions
        ),
        "evidence": 0.0,
        "week": 0,
    },
    schema_overrides={"week": pl.UInt8},
)
WithAllNewActions = simulateActions(initialTab)

/opt/hostedtoolcache/Python/3.11.5/x64/lib/python3.11/site-packages/plotly/express/_core.py:2044: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



# Thompson Sampling

Pega Customer Decision Hub uses a mechanism called Thompson Sampling, which is a method to sample the propensities from a distribution that centers around the model propensity. The width of the distribution depends on the evidence the model has seen. For new actions the propensities are random between 0 and 1. When the model gathers more evidence, the sampled propensities get closer to the model propensities.

This mechanism helps in a few different ways. For outbound channels, when multiple new actions are launched, they will all have the 
same 0.5 propensity (that is where a new model starts). Thompson Sampling effectively adds a bit of noise so it is not always the same
new action that gets selected. For both inbound and outbound channels, Thompson Sampling helps to add some extra "exploration" to the
models in the early phases.

See also:

* https://en.wikipedia.org/wiki/Thompson_sampling
* https://docs.pega.com/pega-customer-decision-hub-user-guide/87/model-learning-new-actions.

## Spread of the Thompson Sampled propensities

The following plot shows how the spread of sampled propensities narrows with more responses
received by the models. We show simulated results for a few different model propensities.

In [3]:
ThompsonSamplingSpread

## Convergence of the Thompson Sampled propensities

To illustrate the fact that the Thompson Sampled propensities quickly converge
to the model propensities, below plot shows the percentage of sampled propensities
that are within 10% of the original model propensity.

For new models (no responses) that is around 50% but at about 200 positive
responses, around 90% of the sampled propensities are within 10% of the 
original model propensities. After 500 positive responses almost all of the
sampled propensities are within that range.

The colors indicate the same base propensities as before, but as can be 
seen, the convergence rate is independent of the base propensities.

In [4]:
PercentageOfSampled


## Beta distribution

In principle, Thompson Sampling is based on the $\beta$-distribution (https://en.wikipedia.org/wiki/Beta_distribution) with "positive" and "negative" responses as the shape parameters. 

But for practical reasons, instead of using $\beta(positives, negatives)$, we approximate this with $\beta(p*evidence, (1-p)*evidence)$.

### Using Positives directly

With https://agilestudio.pega.com/prweb/AgileStudio/app/agilestudio/bugs/BUG-803575 we will
change the implementation in NBAD to use the positives directly so it will become $p + \beta(positives, negatives) - \frac{positives}{evidence}$ with $evidence = positives + negatives$.

### Visualization of Thompson Sampling

Here we show the distribution for a few combinations of model propensity and evidence. Note that the distributions
do not have their top at exactly the model propensity. This is because the beta distribution gives values in the
range 0..1 so for the (typical) small propensity values, the distribution is somewhat right-tailed.

In [5]:
DistributionOfSampled

## Outbound Model Maturity

Thompson Sampling is related to the concept of Outbound Model Maturity. In outbound
channels, we limit the reach of new actions to avoid sending too many actions
"blindly" to customers. As more feedback is accumulated, the percentage of
the customers that can receive the new actions is increased.

We start with 2% of the population for new actions and scale that linearly to
100% until we reach a threshold of 200 positive responses. This is illustrated
by the following plot:

In [6]:
OutboundMaturity

Suppose we start with 5 actions and introduce a new action every week. We
receive feedback after a week, so the reach of the new
actions ramps up after each week until it reaches the 200 positives and is then
mixed with the other actions.

The plot below illustrates how many customers (out of a hypothetical base
of `100000`) get the different actions.

Of course, in an actual implementation, there usually are more factors that are 
considered whether or not to give an action. This includes volume constraints,
propensity thresholding and other factors.

In [7]:
WithExistingActions

## Effect on all new actions

If at day zero all actions are new, it depends on the amount of available
actions.

If there are sufficient actions, then all customers may receive one, but
the targeting will not be very personalized. The distribution will still look even.

If there are few actions, then the model maturity capping may have the effect
that initially not everyone will get an action. As the models mature and become
more targeted, this will change.

In [8]:
WithAllNewActions